In [13]:
from collections import Counter
import nltk
import pandas as pd
import re as regex
import numpy as np
from nltk.tokenize import TweetTokenizer # a tweet tokenizer from nltk.
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

tokenizer = TweetTokenizer()
# stopwords=nltk.corpus.stopwords.words("english")
# whitelist = ["n't", "not"]

In [14]:
def tokenize(tweet):
    def remove_by_regex(tweet,regexp):
        tweet=regex.sub(regexp,"",tweet)
        return tweet
    tweet = str(tweet.lower())
    tweet=remove_by_regex(tweet,regex.compile(r"http.?://[^\s]+[\s]?"))
    tweet=remove_by_regex(tweet,regex.compile(r"@[^\s]+[\s]?"))
    tweet=remove_by_regex(tweet,regex.compile(r"\s?[0-9]+\.?[0-9]*"))
    for remove in map(lambda r: regex.compile(regex.escape(r)), [",", ":", "\"", "=", "&", ";", "%", "$","@", "%", "^", "*", "(", ")", "{", "}","[", "]", "|", "/", "\\", ">", "<", "-","!", "?", ".","--", "---", "#"]):
        tweet=remove_by_regex(tweet,remove)
    tokens=tokenizer.tokenize(tweet)
#     tokens=[w for w in tokens if w in whitelist or not w in stopwords]
    return tokens

In [15]:
def ingest():
    data = pd.read_csv('./tweets.csv',names=['Sentiment','ItemID','Date','Blank','SentimentSource','SentimentText'],encoding="latin-1")
    data.drop(['ItemID', 'SentimentSource'], axis=1, inplace=True)
    data = data[data.Sentiment.isnull() == False]
    data['Sentiment'] = data['Sentiment'].map( {4:1, 0:0} )
    data = data[data['SentimentText'].isnull() == False]
    data.reset_index(inplace=True)
    data.drop('index', axis=1, inplace=True)
    print('dataset loaded with shape', data.shape)   
    return data

In [16]:
data = ingest()
data=data.sample(frac=1).reset_index(drop=True)
da

dataset loaded with shape (1600000, 4)


In [17]:
data.apply(lambda row:[row['Date'],row['Sentiment']])



KeyError: ('Date', 'occurred at index Sentiment')

In [6]:
tokenize(data['SentimentText'][0])

['o', 'how', 'dare', 'they', 'im', 'gonna', 'look', 'on', 'da', 'tinternet']

In [7]:
data['SentimentText'][0]

'@danipoynterjudd :O, how dare they!! im gonna look on da tinternet '

In [8]:
def postprocess(data, n=1000000):
    data = data.head(n)
    data['tokens'] = data['SentimentText'].progress_map(tokenize)  ## progress_map is a variant of the map function plus a progress bar. Handy to monitor DataFrame creations.
    data = data[data.tokens != 'NC']
    data.drop(['Date','SentimentText','Blank'], inplace=True, axis=1)
    data.reset_index(inplace=True)
    data.drop('index', inplace=True, axis=1)
    return data

In [9]:
data = postprocess(data)

progress-bar: 100%|██████████| 1000000/1000000 [08:46<00:00, 1900.98it/s]
/home/nithin/anaconda2/envs/python3/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
data.head()


,Sentiment,tokens
0,1,"[o, how, dare, they, im, gonna, look, on, da, ..."
1,1,"[no, one, was, in, front, of, me, i, took, hun..."
2,0,"[ah, client's, poxy, vpn, failed, again]"
3,0,"[why, aren't, i, at, coachella, i'll, just, wa..."
4,1,"[ta, very, much, just, updating, our, twitter,..."


In [11]:
data.to_pickle("Processedtweets")